In [25]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import word_tokenize
import warnings;
warnings.filterwarnings('ignore')
from collections import Counter 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [35]:
df_raw = pd.read_excel('tnved-CIS-02.xls')
df_raw.drop_duplicates(inplace=True)

In [36]:

STOPWORDS = set(stopwords.words('russian'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s;")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = str(text).lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    


def clean_sentences(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['FULL_TEXT'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df
    
df = clean_sentences(df_raw)

Cleaning sentences...


In [37]:

df.dropna(inplace=True)

In [38]:
df

,TNVED,Cust_ID,Name,Unit,FULL_TEXT,clean_sentence,tok_lem_sentence
1,0101210000,5129.0,––чистопородные племенные животные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ЧИСТОПОРОДНЫЕ ПЛЕМЕННЫЕ ЖИВОТНЫЕ",лошади ослы мулы и лошаки живые лошади чистопородные племенные животные,"[лошади, ослы, мулы, и, лошаки, живые, лошади, чистопородные, племенные, животные]"
3,0101291000,6689.0,–––убойные,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПРОЧИЕ: УБОЙНЫЕ",лошади ослы мулы и лошаки живые лошади прочие убойные,"[лошади, ослы, мулы, и, лошаки, живые, лошади, прочие, убойные]"
4,0101299000,5596.0,–––прочие,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ЛОШАДИ: ПРОЧИЕ: ПРОЧИЕ",лошади ослы мулы и лошаки живые лошади прочие прочие,"[лошади, ослы, мулы, и, лошаки, живые, лошади, прочие, прочие]"
5,0101300000,5117.0,–ослы,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ОСЛЫ",лошади ослы мулы и лошаки живые ослы,"[лошади, ослы, мулы, и, лошаки, живые, ослы]"
6,0101900000,5686.0,–прочие,шт,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ: ПРОЧИЕ",лошади ослы мулы и лошаки живые прочие,"[лошади, ослы, мулы, и, лошаки, живые, прочие]"
...,...,...,...,...,...,...,...
15785,9506620000,5694.0,––мячи надувные,шт,"ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИЗКУЛЬТУРОЙ, ГИМНАСТИКОЙ, ЛЕГКОЙ АТЛЕТИКОЙ, ПРОЧИМИ ВИДАМИ СПОРТА (ВКЛЮЧАЯ НАСТОЛЬНЫЙ ТЕННИС) ИЛИ ДЛЯ ИГР НА ОТКРЫТОМ ВОЗДУХЕ, В ДРУГОМ МЕСТЕ ДАННОЙ ГРУППЫ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ; БАССЕЙНЫ ПЛАВАТЕЛЬН МЯЧИ, КРОМЕ МЯЧЕЙ ДЛЯ ГОЛЬФА И ШАРИКОВ ДЛЯ НАСТОЛЬНОГО ТЕННИСА: МЯЧИ НАДУВНЫЕ",инвентарь и оборудование для занятий общей физкультурой гимнастикой легкой атлетикой прочими видами спорта включая настольный теннис или для игр на открытом воздухе в другом месте данной группы не поименованные или не включенные бассейны плавательн мячи кроме мячей для гольфа и шариков для настольного тенниса мячи надувные,"[инвентарь, и, оборудование, для, занятий, общей, физкультурой, гимнастикой, легкой, атлетикой, прочими, видами, спорта, включая, настольный, теннис, или, для, игр, на, открытом, воздухе, в, другом, месте, данной, группы, не, поименованные, или, не, включенные, бассейны, плавательн, мячи, кроме, мячей, для, гольфа, и, шариков, для, настольного, тенниса, мячи, надувные]"
15787,9506691000,6802.0,–––мячи для крикета и поло,шт,"ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИЗКУЛЬТУРОЙ, ГИМНАСТИКОЙ, ЛЕГКОЙ АТЛЕТИКОЙ, ПРОЧИМИ ВИДАМИ СПОРТА (ВКЛЮЧАЯ НАСТОЛЬНЫЙ ТЕННИС) ИЛИ ДЛЯ ИГР НА ОТКРЫТОМ ВОЗДУХЕ, В ДРУГОМ МЕСТЕ ДАННОЙ ГРУППЫ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ; БАССЕЙНЫ ПЛАВАТЕЛЬН МЯЧИ, КРОМЕ МЯЧЕЙ ДЛЯ ГОЛЬФА И ШАРИКОВ ДЛЯ НАСТОЛЬНОГО ТЕННИСА: ПРОЧИЕ: МЯЧИ ДЛЯ КРИКЕТА И ПОЛО",инвентарь и оборудование для занятий общей физкультурой гимнастикой легкой атлетикой прочими видами спорта включая настольный теннис или для игр на открытом воздухе в другом месте данной группы не поименованные или не включенные бассейны плавательн мячи кроме мячей для гольфа и шариков для настольного тенниса прочие мячи для крикета и поло,"[инвентарь, и, оборудование, для, занятий, общей, физкультурой, гимнастикой, легкой, атлетикой, прочими, видами, спорта, включая, настольный, теннис, или, для, игр, на, открытом, воздухе, в, другом, месте, данной, группы, не, поименованные, или, не, включенные, бассейны, плавательн, мячи, кроме, мячей, для, гольфа, и, шариков, для, настольного, тенниса, прочие, мячи, для, крикета, и, поло]"
15788,9506699000,5291.0,–––прочие,шт,"ИНВЕНТАРЬ И ОБОРУДОВАНИЕ ДЛЯ ЗАНЯТИЙ ОБЩЕЙ ФИЗКУЛЬТУРОЙ, ГИМНАСТИКОЙ, ЛЕГКОЙ АТЛЕТИКОЙ, ПРОЧИМИ ВИДАМИ СПОРТА (ВКЛЮЧАЯ НАСТОЛЬНЫЙ ТЕННИС) ИЛИ ДЛЯ ИГР НА ОТКРЫТОМ ВОЗДУХЕ, В ДРУГОМ МЕСТЕ ДАННОЙ ГРУППЫ НЕ ПОИМЕНОВАННЫЕ ИЛИ НЕ ВКЛЮЧЕННЫЕ; БАССЕЙНЫ ПЛАВАТЕЛЬН МЯЧИ, КРОМЕ МЯЧЕЙ ДЛЯ ГОЛЬФА И ШАРИКОВ ДЛЯ НАСТОЛЬНОГО ТЕННИСА: ПРОЧИЕ: ПРОЧИЕ",инвентарь и оборудование для занятий общей физкультурой гимнастикой легкой атлетикой прочими видами спорта включая настольный теннис или для игр на открытом воздухе в другом месте данной группы не поименованные или не включенные бассейны плавательн мячи кроме мячей д

In [39]:
def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Adapt stop words
token_stop = tokenizer(' '.join(STOPWORDS), lemmatize=False)

# Fit TFIDF
vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer) 
tfidf_mat = vectorizer.fit_transform(df['clean_sentence'].values) # -> (num_sentences, num_vocabulary)
tfidf_mat.shape

(11454, 12316)

In [107]:
query_sentence='АНТИКВАРИАТ ВОЗРАСТОМ БОЛЕЕ 100 ЛЕТ, Россия. Москва, 210x250см, На выставку "Выставка мамы Кайрата"'

In [108]:
def get_recommendations_tfidf(sentence, tfidf_mat):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    # Embed the query sentence
    tokens = [str(tok) for tok in tokenizer(sentence)]
    vec = vectorizer.transform(tokens)
    # Create list with similarity between query and dataset
    mat = cosine_similarity(vec, tfidf_mat)
    # Best cosine distance for each token independantly
    print(mat.shape)
    best_index = extract_best_indices(mat, topk=10)
    return best_index


best_index = get_recommendations_tfidf(query_sentence, tfidf_mat)

display(df[['TNVED', 'Name', 'FULL_TEXT']].iloc[best_index])


(19, 11454)


,TNVED,Name,FULL_TEXT
14560,8704329905,"––––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА ДЛЯ ПЕРЕВОЗКИ ГРУЗОВ: ПРОЧИЕ, С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИГАНИЕМ: С ПОЛНОЙ МАССОЙ ТРАНСПОРТНОГО СРЕДСТВА БОЛЕЕ 5 Т: ПРОЧИЕ: БЫВШИЕ В ЭКСПЛУАТАЦИИ: ПРОЧИЕ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14373,8702901995,"––––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА, ПРЕДНАЗНАЧЕННЫЕ ДЛЯ ПЕРЕВОЗКИ 10 ЧЕЛОВЕК ИЛИ БОЛЕЕ, ВКЛЮЧАЯ ВОДИТЕЛЯ: ПРОЧИЕ: С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИГАНИЕМ: С РАБОЧИМ ОБЪЕМОМ ЦИЛИНДРОВ ДВИГАТЕЛЯ БОЛЕЕ 2800 СМ³: БЫВШИЕ В ЭКСПЛУАТАЦИИ: ПРОЧИЕ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14387,8702903994,"––––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА, ПРЕДНАЗНАЧЕННЫЕ ДЛЯ ПЕРЕВОЗКИ 10 ЧЕЛОВЕК ИЛИ БОЛЕЕ, ВКЛЮЧАЯ ВОДИТЕЛЯ: ПРОЧИЕ: С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИГАНИЕМ: С РАБОЧИМ ОБЪЕМОМ ЦИЛИНДРОВ ДВИГАТЕЛЯ НЕ БОЛЕЕ 2800 СМ³: БЫВШИЕ В ЭКСПЛУАТАЦИИ: ПРОЧИЕ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14538,8704239905,"–––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА ДЛЯ ПЕРЕВОЗКИ ГРУЗОВ: ПРОЧИЕ, С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ВОСПЛАМЕНЕНИЕМ ОТ СЖАТИЯ (ДИЗЕЛЕМ ИЛИ ПОЛУДИЗЕЛЕМ): С ПОЛНОЙ МАССОЙ ТРАНСПОРТНОГО СРЕДСТВА БОЛЕЕ 20 Т: ПРОЧИЕ: БЫВШИЕ В ЭКСПЛУАТАЦИИ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14305,8701209015,"––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","ТРАКТОРЫ (КРОМЕ ТРАКТОРОВ ТОВАРНОЙ ПОЗИЦИИ 8709): ТРАКТОРЫ КОЛЕСНЫЕ ДЛЯ ПОЛУПРИЦЕПОВ: БЫВШИЕ В ЭКСПЛУАТАЦИИ: ТЯГАЧИ СЕДЕЛЬНЫЕ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14340,8702101995,"–––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА, ПРЕДНАЗНАЧЕННЫЕ ДЛЯ ПЕРЕВОЗКИ 10 ЧЕЛОВЕК ИЛИ БОЛЕЕ, ВКЛЮЧАЯ ВОДИТЕЛЯ: С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ВОСПЛАМЕНЕНИЕМ ОТ СЖАТИЯ (ДИЗЕЛЕМ ИЛИ ПОЛУДИЗЕЛЕМ): С РАБОЧИМ ОБЪЕМОМ ЦИЛИНДРОВ ДВИГАТЕЛЯ БОЛЕЕ 2500 СМ³: БЫВШИЕ В ЭКСПЛУАТАЦИИ: ПРОЧИЕ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14357,8702109994,"–––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА, ПРЕДНАЗНАЧЕННЫЕ ДЛЯ ПЕРЕВОЗКИ 10 ЧЕЛОВЕК ИЛИ БОЛЕЕ, ВКЛЮЧАЯ ВОДИТЕЛЯ: С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ВОСПЛАМЕНЕНИЕМ ОТ СЖАТИЯ (ДИЗЕЛЕМ ИЛИ ПОЛУДИЗЕЛЕМ): С РАБОЧИМ ОБЪЕМОМ ЦИЛИНДРОВ ДВИГАТЕЛЯ НЕ БОЛЕЕ 2500 СМ³: БЫВШИЕ В ЭКСПЛУАТАЦИИ: ПРОЧИЕ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14545,8704313904,"––––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА ДЛЯ ПЕРЕВОЗКИ ГРУЗОВ: ПРОЧИЕ, С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИГАНИЕМ: С ПОЛНОЙ МАССОЙ ТРАНСПОРТНОГО СРЕДСТВА НЕ БОЛЕЕ 5 Т: ПРОЧИЕ: С РАБОЧИМ ОБЪЕМОМ ЦИЛИНДРОВ ДВИГАТЕЛЯ БОЛЕЕ 2800 СМ³: БЫВШИЕ В ЭКСПЛУАТАЦИИ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14550,8704319904,"––––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА ДЛЯ ПЕРЕВОЗКИ ГРУЗОВ: ПРОЧИЕ, С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ИСКРОВЫМ ЗАЖИГАНИЕМ: С ПОЛНОЙ МАССОЙ ТРАНСПОРТНОГО СРЕДСТВА НЕ БОЛЕЕ 5 Т: ПРОЧИЕ: С РАБОЧИМ ОБЪЕМОМ ЦИЛИНДРОВ ДВИГАТЕЛЯ НЕ БОЛЕЕ 2800 СМ³: БЫВШИЕ В ЭКСПЛУАТАЦИИ: С МОМЕНТА ВЫПУСКА КОТОРЫХ ПРОШЛО БОЛЕЕ 5 ЛЕТ, НО НЕ БОЛЕЕ 7 ЛЕТ"
14528,8704229905,"––––––с момента выпуска которых прошло более 5 лет, но не более 7 лет","МОТОРНЫЕ ТРАНСПОРТНЫЕ СРЕДСТВА ДЛЯ ПЕРЕВОЗКИ ГРУЗОВ: ПРОЧИЕ, С ПОРШНЕВЫМ ДВИГАТЕЛЕМ ВНУТРЕННЕГО СГОРАНИЯ С ВОСПЛАМЕНЕНИЕМ ОТ СЖАТИЯ (ДИЗЕЛЕМ ИЛИ ПОЛУДИЗЕЛЕМ): С ПОЛНОЙ МАССОЙ ТРАНСПОРТНОГО СРЕДСТВА БОЛЕЕ 5 Т, НО НЕ БОЛЕЕ 20 Т: ПРОЧИЕ: БЫВШИЕ В ЭКСПЛУАТАЦИИ: ПРОЧИЕ: С МОМЕНТА ВЫ